<a href="https://colab.research.google.com/github/nitishast/TF/blob/master/Image_Classifier_Using_Sequential_API_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import pandas as pd
import matplotlib as mp
import tensorflow as tf
from tensorflow import keras

In [3]:
keras.__version__

'2.2.4-tf'

### Loading the MNIST Dataset

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

### In scikit learn same image is represented as 1D array of 784 but in keras each is 28x28 array

In [5]:
(X_train_full,y_train_full),(X_test,y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
print (X_train_full.shape)
print (X_train_full.dtype)

(60000, 28, 28)
uint8


### Since we will optimise the network by using Gradient descent, we must scale the input feature to 0-1. Can be done by dividing by max feature 255

In [0]:
# Creating a Validation set 
X_valid, X_train = X_train_full[:5000]/255.0 , X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000]/255.0 , y_train_full[5000:]

In [0]:
class_names = ["T-shirt/top" , "Trouser" , " Pullover", "Dress" , "Coat" , "Sandal" , "Shirt" , "Sneaker" , "Bag" , "Ankle boot"]

In [9]:
class_names[y_train[0]]

'Coat'

## Building a sequential model

In [10]:
from keras.models import Sequential
from keras.layers import Flatten,Dense

Using TensorFlow backend.


#### The shape of the weight matrix depends on the number of inputs.
This is why it is recommended to specify the input_shape when
creating the first layer in a Sequential model. However, if you do
not specify the input shape, it’s okay: Keras will simply wait until it
knows the input shape before it actually builds the model. This will
happen either when you feed it actual data (e.g., during training),
or when you call its build() method. Until the model is really
built, the layers will not have any weights, and you will not be able
to do certain things (such as print the model summary or save the
model), so if you know the input shape when creating the model, it
is best to specify it

In [0]:
model = keras.models.Sequential()
#Input Layer
model.add(keras.layers.Flatten(input_shape=[28,28])) # Just flattens the image vector into a 1D array of 784 by computing X.reshape(-1,1)
#Fully connected layers -Dense
model.add(keras.layers.Dense(300,activation="relu"))
model.add(keras.layers.Dense(100,activation="relu"))

#Output layer
model.add(keras.layers.Dense(10,activation="softmax")) #10 neurons -- 1 neuron per class. Siince there are 10 class hence 10 neurons



In [12]:
model.summary() ## Here None is the batch_size which it can be anything.
## Param is the number of trainable number of parameters in the layer
## 1st dense layer has (784[input shape]x300[neurons]) connection weights (+) 300 bias term[one bais term per neuron] == 235500
## 2nd dense layer has (300x100) + 100 bais terms = 30100 params
## 3rd dense layer has (100x10) + 10 bias term == 1010 params

#------------>> Lots of flexibility also means higher risk of overfitting especially if training data is less.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


## Getting weights and bias for each layers

In [13]:
model.layers[1].name

'dense'

In [0]:
weights, bias = model.layers[1].get_weights()

In [15]:
weights


array([[ 0.0282152 , -0.04315449, -0.00720893, ...,  0.01632708,
        -0.01088278,  0.01853196],
       [-0.00802235, -0.0508032 , -0.05854437, ...,  0.03820658,
        -0.06983528, -0.03277129],
       [ 0.03903367,  0.05755739, -0.05311631, ...,  0.02698725,
         0.06070229,  0.02414948],
       ...,
       [-0.02486024, -0.04750348, -0.00920478, ..., -0.02345226,
        -0.03750836,  0.06642896],
       [-0.01798847, -0.00153072,  0.01298897, ...,  0.06312975,
        -0.00620316,  0.01717275],
       [ 0.01835575, -0.00866996,  0.06963141, ...,  0.0174543 ,
        -0.07124001,  0.01627947]], dtype=float32)

In [16]:
print (weights.shape)
print (bias.shape)

(784, 300)
(300,)


### In compile method we define the loss methods like cross-entropy for classification and MSE/Huber for regression tasks. Also we need to specify the optimizer and the metrics

#### Loss:

> Sparse crossentropy loss because we the labels to predict are sparse, and exclusive. if the labels to predict we binary categorical_crossentropy loss.

Activiation:

> Softmax because multiclass classification. if we were using Binary loss then sigmoid

Optimizer:

>Different type of optimizer can be used. SGD simply means stociastic gradient descent for backpropogation. 

Metric:

>Accuracy as this is a classification task



In [0]:
model.compile(optimizer='sgd',
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [18]:
## Epocs of 1 will not be sufficient for convergence hence use the epocs suitable.
model.fit(X_train,y_train, epochs=30,validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 7s 125us/sample - loss: 0.7154 - accuracy: 0.7656 - val_loss: 6.1602 - val_accuracy: 0.0778
Epoch 2/30
55000/55000 [==============================] - 5s 91us/sample - loss: 0.4895 - accuracy: 0.8293 - val_loss: 6.7518 - val_accuracy: 0.0764
Epoch 3/30
55000/55000 [==============================] - 5s 91us/sample - loss: 0.4442 - accuracy: 0.8443 - val_loss: 7.1336 - val_accuracy: 0.0766
Epoch 4/30
55000/55000 [==============================] - 5s 92us/sample - loss: 0.4175 - accuracy: 0.8539 - val_loss: 6.8946 - val_accuracy: 0.0806
Epoch 5/30
55000/55000 [==============================] - 5s 93us/sample - loss: 0.3972 - accuracy: 0.8606 - val_loss: 7.3509 - val_accuracy: 0.0786
Epoch 6/30
55000/55000 [==============================] - 5s 96us/sample - loss: 0.3814 - accuracy: 0.8658 - val_loss: 7.7329 - val_accuracy: 0.0746
Epoch 7/30
55000/55000 [==============================] 